<a href="https://colab.research.google.com/github/gondore/nsdc-crafter/blob/main/experimental_dqn_nsdc_crafter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
pip install gymnasium crafter shimmy[gymv21] stable_baselines3 shimmy[gym-v26]

  Using cached gym-0.26.2-py3-none-any.whl
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.0.6 requires gym<=0.25.2, but you have gym 0.26.2 which is incompatible.


In [47]:
pip install shimmy[gym-v21]

  Using cached gym-0.25.2-py3-none-any.whl
  Attempting uninstall: gym
    Found existing installation: gym 0.26.2
    Uninstalling gym-0.26.2:
      Successfully uninstalled gym-0.26.2


In [48]:
import gymnasium as gym
import crafter


#env = gym.make("GymV26Environment-v0", env_id = 'CrafterReward-v1')  # Or CrafterNoReward-v1
env = crafter.Env()
env = crafter.Recorder(
  env, './path/to/logdir',
  save_stats=True,
  save_video=False,
  save_episode=False,
)
import matplotlib.pyplot as plt
import numpy as np

It looks like our observation space is image data? 64x64x3 (height x width x channel)

In [49]:
print("Observation Space", env.observation_space)


Observation Space Box(0, 255, (64, 64, 3), uint8)


In [50]:
observation_space = env.observation_space

According to crafter documentation, (https://github.com/danijar/crafter/blob/e04542a2159f1aad3d4c5ad52e8185717380ee3a/crafter/env.py#L15)

```
  def observation_space(self):
    return BoxSpace(0, 255, tuple(self._size) + (3,), np.uint8)
```

 I believe the observation space is in the form of BoxSpace, which I'm not familiar with. This plot below is just a quick script that visualizes the observation space. I think we will have to work with image data and flatten it to state representations?


In [51]:
import tensorflow as tf      # Deep Learning library
import numpy as np           # Handle matrices


from skimage import transform # Help us to preprocess the frames
from skimage.color import rgb2gray # Help us to gray our frames

import matplotlib.pyplot as plt # Display graphs

from collections import deque# Ordered collection with ends

import random

import warnings # This ignore all the warning messages that are normally printed during the training because of skiimage
warnings.filterwarnings('ignore')

In [52]:
print("The action size is : ", env.action_space.n)
# one hot encode actions
possible_actions = np.array(np.identity(env.action_space.n,dtype=int).tolist())


The action size is :  17


**Preprocessing observationspace**
- grayscale frames
- normalizing pixel values
- resize preprocessed frame


In [62]:
def preprocess_frame(frame):
    # Greyscale frame
    gray = rgb2gray(frame)

    # Normalize Pixel Values
    normalized_frame = frame/255.0

    # Resize
    # Thanks to Mikołaj Walkowiak
    preprocessed_frame = transform.resize(normalized_frame, [110,84])

    return preprocessed_frame # 110x84x1 frame

**stack_frames**

In [54]:
stack_size = 4 # We stack 4 frames

# Initialize deque with zero-images one array for each image
stacked_frames  =  deque([np.zeros((110,84), dtype=int) for i in range(stack_size)], maxlen=4)

def stack_frames(stacked_frames, state, is_new_episode):
    # Preprocess frame
    frame = preprocess_frame(state)

    if is_new_episode:
        # Clear our stacked_frames
        stacked_frames = deque([np.zeros((110,84), dtype=int) for i in range(stack_size)], maxlen=4)

        # Because we're in a new episode, copy the same frame 4x
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)

        # Stack the frames
        stacked_state = np.stack(stacked_frames, axis=2)

    else:
        # Append frame to deque, automatically removes the oldest frame
        stacked_frames.append(frame)

        # Build the stacked state (first dimension specifies different frames)
        stacked_state = np.stack(stacked_frames, axis=2)

    return stacked_state, stacked_frames

**Hyperparameter setup**

In [55]:
### MODEL HYPERPARAMETERS
state_size = [110, 84, 4]      # Our input is a stack of 4 frames hence 110x84x4 (Width, height, channels)
action_size = env.action_space.n # 8 possible actions
learning_rate =  0.00025      # Alpha (aka learning rate)

### TRAINING HYPERPARAMETERS
total_episodes = 50            # Total episodes for training
max_steps = 50000              # Max possible steps in an episode
batch_size = 64                # Batch size

# Exploration parameters for epsilon greedy strategy
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability
decay_rate = 0.00001           # exponential decay rate for exploration prob

# Q learning hyperparameters
gamma = 0.9                    # Discounting rate

### MEMORY HYPERPARAMETERS
pretrain_length = batch_size   # Number of experiences stored in the Memory when initialized for the first time
memory_size = 1000000          # Number of experiences the Memory can keep

### PREPROCESSING HYPERPARAMETERS
stack_size = 4                 # Number of frames stacked

### MODIFY THIS TO FALSE IF YOU JUST WANT TO SEE THE TRAINED AGENT
training = False

## TURN THIS TO TRUE IF YOU WANT TO RENDER THE ENVIRONMENT
episode_render = False

**DQN implementation**

In [56]:
class DQNetwork:
    def __init__(self, state_size, action_size, learning_rate, name='DQNetwork'):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate

        with tf.compat.v1.variable_scope(name):
            # We create the placeholders
            # *state_size means that we take each element of state_size in tuple hence is like if we wrote
            # [None, 84, 84, 4]
            self.inputs_ = tf.keras.Input(shape=state_size, dtype=tf.float32, name="inputs")
            self.actions_ = tf.keras.Input(shape=(self.action_size,), dtype=tf.float32, name="actions_")

            # Remember that target_Q is the R(s,a) + ymax Qhat(s', a')
            self.target_Q = tf.keras.Input(shape=(1,), dtype=tf.float32, name="target")

            """
            First convnet:
            CNN
            ELU
            """
            # Input is 110x84x4
            self.conv1 = tf.keras.layers.Conv2D(filters=32,
                                                kernel_size=(8, 8),
                                                strides=(4, 4),
                                                padding="VALID",
                                                kernel_initializer=tf.initializers.GlorotUniform(),
                                                name="conv1")
            self.conv1_out = tf.keras.layers.ELU(name="conv1_out")

            """
            Second convnet:
            CNN
            ELU
            """
            self.conv2 = tf.keras.layers.Conv2D(filters=64,
                                                kernel_size=(4, 4),
                                                strides=(2, 2),
                                                padding="VALID",
                                                kernel_initializer=tf.initializers.GlorotUniform(),
                                                name="conv2")
            self.conv2_out = tf.keras.layers.ELU(name="conv2_out")

            """
            Third convnet:
            CNN
            ELU
            """
            self.conv3 = tf.keras.layers.Conv2D(filters=64,
                                                kernel_size=(3, 3),
                                                strides=(2, 2),
                                                padding="VALID",
                                                kernel_initializer=tf.initializers.GlorotUniform(),
                                                name="conv3")
            self.conv3_out = tf.keras.layers.ELU(name="conv3_out")

            self.flatten = tf.keras.layers.Flatten()
            self.fc = tf.keras.layers.Dense(units=512,
                                             activation=tf.keras.activations.elu,
                                             kernel_initializer=tf.initializers.GlorotUniform(),
                                             name="fc1")
            self.output = tf.keras.layers.Dense(units=self.action_size,
                                                 activation=None,
                                                 kernel_initializer=tf.initializers.GlorotUniform(),
                                                 name="output")

            # Connect the layers
            x = self.conv1(self.inputs_)
            x = self.conv1_out(x)
            x = self.conv2(x)
            x = self.conv2_out(x)
            x = self.conv3(x)
            x = self.conv3_out(x)
            x = self.flatten(x)
            x = self.fc(x)
            self.output_ = self.output(x)

            # Q is our predicted Q value.
            self.Q = tf.reduce_sum(tf.multiply(self.output_, self.actions_))

            # The loss is the difference between our predicted Q_values and the Q_target
            # Sum(Qtarget - Q)^2
            self.loss = tf.reduce_mean(tf.square(self.target_Q - self.Q))

            # Get the list of trainable variables
            self.trainable_variables = self.conv1.trainable_variables + \
                                        self.conv1_out.trainable_variables + \
                                        self.conv2.trainable_variables + \
                                        self.conv2_out.trainable_variables + \
                                        self.conv3.trainable_variables + \
                                        self.conv3_out.trainable_variables + \
                                        self.fc.trainable_variables + \
                                        self.output.trainable_variables

            # Create an optimizer
            self.optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)

    def train(self, inputs, actions, target_Q):
        with tf.GradientTape() as tape:
            # Forward pass
            predicted_Q = self(inputs)
            # Calculate the loss
            loss = tf.reduce_mean(tf.square(target_Q - predicted_Q))

        # Get gradients of loss w.r.t. the trainable variables
        gradients = tape.gradient(loss, self.trainable_variables)
        # Apply gradients using the optimizer
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return loss


In [57]:
# Reset the graph
tf.compat.v1.reset_default_graph()
# Instantiate the DQNetwork
DQNetwork = DQNetwork(state_size, action_size, learning_rate)

In [58]:
class Memory():
    def __init__(self, max_size):
        self.buffer = deque(maxlen = max_size)

    def add(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        buffer_size = len(self.buffer)
        index = np.random.choice(np.arange(buffer_size),
                                size = batch_size,
                                replace = False)

        return [self.buffer[i] for i in index]

In [68]:
# Instantiate memory
memory = Memory(max_size = memory_size)
for i in range(pretrain_length):
    # If it's the first step
    if i == 0:
        state = env.reset()

        state, stacked_frames = stack_frames(stacked_frames, state, True)

    # Get the next_state, the rewards, done by taking a random action
    choice = random.randint(1,len(possible_actions))-1
    action = possible_actions[choice]
    next_state, reward, done, _ = env.step(np.argmax(action))

    #env.render()

    # Stack the frames
    next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)


    # If the episode is finished (we're dead 3x)
    if done:
        # We finished the episode
        next_state = np.zeros(state.shape)

        # Add experience to memory
        memory.add((state, action, reward, next_state, done))

        # Start a new episode
        state = env.reset()

        # Stack the frames
        state, stacked_frames = stack_frames(stacked_frames, state, True)

    else:
        # Add experience to memory
        memory.add((state, action, reward, next_state, done))

        # Our new state is now the next_state
        state = next_state

In [71]:
tensorboard --logdir=/tensorboard/dqn/1

SyntaxError: cannot assign to expression (<ipython-input-71-dc589fdfe3b7>, line 1)

In [70]:
# Setup TensorBoard Writer
writer = tf.summary.FileWriter("/tensorboard/dqn/1")

## Losses
tf.summary.scalar("Loss", DQNetwork.loss)

write_op = tf.summary.merge_all()

AttributeError: module 'tensorboard.summary._tf.summary' has no attribute 'FileWriter'

In [72]:
def predict_action(explore_start, explore_stop, decay_rate, decay_step, state, actions):
    ## EPSILON GREEDY STRATEGY
    # Choose action a from state s using epsilon greedy.
    ## First we randomize a number
    exp_exp_tradeoff = np.random.rand()

    # Here we'll use an improved version of our epsilon greedy strategy used in Q-learning notebook
    explore_probability = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * decay_step)

    if (explore_probability > exp_exp_tradeoff):
        # Make a random action (exploration)
        choice = random.randint(1,len(possible_actions))-1
        action = possible_actions[choice]

    else:
        # Get action from Q-network (exploitation)
        # Estimate the Qs values state
        Qs = sess.run(DQNetwork.output, feed_dict = {DQNetwork.inputs_: state.reshape((1, *state.shape))})

        # Take the biggest Q value (= the best action)
        choice = np.argmax(Qs)
        action = possible_actions[choice]


    return action, explore_probability

In [78]:
# Saver will help us to save our model
#saver = tf.train.Saver()

if training == True:


    with tf.Session() as sess:
        # Initialize the variables
        sess.run(tf.global_variables_initializer())

        # Initialize the decay rate (that will use to reduce epsilon)
        decay_step = 0

        for episode in range(total_episodes):
            # Set step to 0
            step = 0

            # Initialize the rewards of the episode
            episode_rewards = []

            # Make a new episode and observe the first state
            state = env.reset()

            # Remember that stack frame function also call our preprocess function.
            state, stacked_frames = stack_frames(stacked_frames, state, True)

            while step < max_steps:
                step += 1

                #Increase decay_step
                decay_step +=1

                # Predict the action to take and take it
                action, explore_probability = predict_action(explore_start, explore_stop, decay_rate, decay_step, state, possible_actions)

                #Perform the action and get the next_state, reward, and done information
                next_state, reward, done, _ = env.step(action)

                if episode_render:
                    env.render()

                # Add the reward to total reward
                episode_rewards.append(reward)

                # If the game is finished
                if done:
                    # The episode ends so no next state
                    next_state = np.zeros((110,84), dtype=np.int)

                    next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)

                    # Set step = max_steps to end the episode
                    step = max_steps

                    # Get the total reward of the episode
                    total_reward = np.sum(episode_rewards)

                    print('Episode: {}'.format(episode),
                                  'Total reward: {}'.format(total_reward),
                                  'Explore P: {:.4f}'.format(explore_probability),
                                'Training Loss {:.4f}'.format(loss))

                    rewards_list.append((episode, total_reward))

                    # Store transition <st,at,rt+1,st+1> in memory D
                    memory.add((state, action, reward, next_state, done))

                else:
                    # Stack the frame of the next_state
                    next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)

                    # Add experience to memory
                    memory.add((state, action, reward, next_state, done))

                    # st+1 is now our current state
                    state = next_state


                ### LEARNING PART
                # Obtain random mini-batch from memory
                batch = memory.sample(batch_size)
                states_mb = np.array([each[0] for each in batch], ndmin=3)
                actions_mb = np.array([each[1] for each in batch])
                rewards_mb = np.array([each[2] for each in batch])
                next_states_mb = np.array([each[3] for each in batch], ndmin=3)
                dones_mb = np.array([each[4] for each in batch])

                target_Qs_batch = []

                # Get Q values for next_state
                Qs_next_state = sess.run(DQNetwork.output, feed_dict = {DQNetwork.inputs_: next_states_mb})

                # Set Q_target = r if the episode ends at s+1, otherwise set Q_target = r + gamma*maxQ(s', a')
                for i in range(0, len(batch)):
                    terminal = dones_mb[i]

                    # If we are in a terminal state, only equals reward
                    if terminal:
                        target_Qs_batch.append(rewards_mb[i])

                    else:
                        target = rewards_mb[i] + gamma * np.max(Qs_next_state[i])
                        target_Qs_batch.append(target)


                targets_mb = np.array([each for each in target_Qs_batch])

                loss, _ = sess.run([DQNetwork.loss, DQNetwork.optimizer],
                                        feed_dict={DQNetwork.inputs_: states_mb,
                                                   DQNetwork.target_Q: targets_mb,
                                                   DQNetwork.actions_: actions_mb})

                # Write TF Summaries
                summary = sess.run(write_op, feed_dict={DQNetwork.inputs_: states_mb,
                                                       DQNetwork.target_Q: targets_mb,
                                                       DQNetwork.actions_: actions_mb})
                writer.add_summary(summary, episode)
                writer.flush()

            # Save model every 5 episodes
            if episode % 5 == 0:
                save_path = model.save("./models/model.ckpt")
                print("Model Saved")